# Closed-Loop Evaluation
In this notebook you are going to evaluate Urban Driver to control the SDV with a protocol named *closed-loop* evaluation.

**Note: this notebook assumes you've already run the [training notebook](./train.ipynb) and stored your model successfully (or that you have stored a pre-trained one).**

**Note: for a detailed explanation of what closed-loop evaluation (CLE) is, please refer to our [planning notebook](../planning/closed_loop_test.ipynb)**

### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from prettytable import PrettyTable

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset

from l5kit.dataset import EgoDatasetVectorized
from l5kit.vectorization.vectorizer_builder import build_vectorizer

from l5kit.rasterization.rasterizer_builder import build_rasterizer
from l5kit.simulation.dataset import SimulationConfig
from l5kit.simulation.unroll import ClosedLoopSimulator
from l5kit.cle.closed_loop_evaluator import ClosedLoopEvaluator, EvaluationPlan
from l5kit.cle.metrics import (CollisionFrontMetric, CollisionRearMetric, CollisionSideMetric,
                               DisplacementErrorL2Metric, DistanceToRefTrajectoryMetric)
from l5kit.cle.validators import RangeValidator, ValidationCountingAggregator

from l5kit.visualization.visualizer.zarr_utils import simulation_out_to_visualizer_scene
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.data import MapAPI

from collections import defaultdict
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import sys
from pathlib import Path
sys.path.append('/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/scripts')
sys.path.append('/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/scripts/vectorized_offline_rl_model')
sys.path.append('/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/scripts/reward')
import vectorized_offline_rl_model
from vectorized_offline_rl_model import VectorOfflineRLModel, EnsembleOfflineRLModel

from l5kit.planning.vectorized.closed_loop_model import VectorizedUnrollModel  
from l5kit.planning.vectorized.open_loop_model import VectorizedModel

project path:  /mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl


## Prepare data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params ...).

In [2]:
project_path = "/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl"
print("project path: ", project_path)
sys.path.append(project_path)

# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "/mnt/share_disk/user/public/l5kit/prediction"
dm = LocalDataManager(None)
# get config
cfg = load_config_data(str(Path(project_path, "scripts/offline_rl_config.yaml")))
print(cfg)

project path:  /mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl
{'format_version': 7, 'model_params': {'history_num_frames_ego': 1, 'history_num_frames': 0, 'history_num_frames_agents': 3, 'step_time': 0.1, 'disable_other_agents': False, 'disable_map': False, 'disable_lane_boundaries': True, 'global_head_dropout': 0.0, 'future_num_frames': 12, 'detach_unroll': True, 'warmup_num_frames': 0, 'discount_factor': 0.8, 'render_ego_history': True}, 'train_data_loader': {'key': 'scenes/train.zarr', 'batch_size': 16, 'pred_len': 10, 'shuffle': True, 'num_workers': 2, 'perturb_probability': 0.5, 'yaml': None}, 'val_data_loader': {'key': 'scenes/train.zarr', 'batch_size': 16, 'shuffle': False, 'num_workers': 2}, 'raster_params': {'raster_size': [224, 224], 'pixel_size': [0.5, 0.5], 'ego_center': [0.5, 0.5], 'map_type': 'py_satellite', 'satellite_map_key': 'aerial_map/aerial_map.png', 'semantic_map_key': 'semantic_map/semantic_map.pb', 'dataset_meta_key': 'meta.json', 'filter_agents_thre

## Load the model

In [3]:
model_name = "Offline RL Planner"
weights_scaling = [1.0, 1.0, 1.0]
_num_predicted_frames = cfg["model_params"]["future_num_frames"]
_num_predicted_params = len(weights_scaling)
kwargs=dict(
            history_num_frames_ego=cfg["model_params"]["history_num_frames_ego"],
            history_num_frames_agents=cfg["model_params"]["history_num_frames_agents"],
            num_targets=_num_predicted_params * _num_predicted_frames,
            weights_scaling=weights_scaling,
            criterion=torch.nn.L1Loss(reduction="none"),
            global_head_dropout=cfg["model_params"]["global_head_dropout"],
            disable_other_agents=cfg["model_params"]["disable_other_agents"],
            disable_map=cfg["model_params"]["disable_map"],
            disable_lane_boundaries=cfg["model_params"]["disable_lane_boundaries"],
            cfg=cfg
)


num_ensemble = 2
model_list = [VectorOfflineRLModel(**kwargs) for _ in range(num_ensemble)]

model_path0="/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/tmpperturb_2_13_0.3/Offline RL Planner-train_flag_0signal_scene_13-il_weight_1.0-pred_weight_1.0-pretrained_True-1/iter_0040000.pt"
model_path1="/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/tmpperturb_2_13_0.3/Offline RL Planner-train_flag_0signal_scene_13-il_weight_1.0-pred_weight_1.0-pretrained_True-1/iter_0040000.pt"
model_path2="/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/tmphistory_10_perturb_2_1000_0.3/Offline RL Planner-train_flag_0signal_scene_0-il_weight_1.0-pred_weight_1.0-pretrained_True-1/iter_0020000.pt"
model_path3="/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/tmphistory_10_perturb_2_1000_0.3/Offline RL Planner-train_flag_0signal_scene_0-il_weight_1.0-pred_weight_1.0-pretrained_True-1/iter_0020000.pt"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_list[0].load_state_dict(torch.load(model_path0))
model_list[1].load_state_dict(torch.load(model_path1))
# model_list[2].load_state_dict(torch.load(model_path2))
# model_list[3].load_state_dict(torch.load(model_path3))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = EnsembleOfflineRLModel(model_list)
model=model.to(device)
model = model.eval()
torch.set_grad_enabled(False)

In [ ]:
model_name = "Offline RL Planner"
num_ensemble = 4
model_list = [model for _ in range(num_ensemble)]
model = EnsembleOfflineRLModel(model_list)

In [ ]:
# mbop
model_path="/mnt/share_disk/user/daixingyuan/l5kit/tmp/Offline RL Planner-signal_scene_13-il_weight_1.0-pred_weight_1.0-1/iter_0025000.pt"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_path))
model=model.to(device)
model = model.eval()
torch.set_grad_enabled(False)

In [ ]:
#baseline urban_driver  toch_static
weights_scaling = [1.0, 1.0, 1.0]
_num_predicted_frames = cfg["model_params"]["future_num_frames"]
_num_predicted_params = len(weights_scaling)
import torch.nn as nn

model = VectorizedUnrollModel(
    history_num_frames_ego=cfg["model_params"]["history_num_frames_ego"],
    history_num_frames_agents=cfg["model_params"]["history_num_frames_agents"],
    num_targets=_num_predicted_params * _num_predicted_frames,
    weights_scaling=weights_scaling,
    criterion=nn.L1Loss(reduction="none"),
    global_head_dropout=cfg["model_params"]["global_head_dropout"],
    disable_other_agents=cfg["model_params"]["disable_other_agents"],
    disable_map=cfg["model_params"]["disable_map"],
    disable_lane_boundaries=cfg["model_params"]["disable_lane_boundaries"],
    detach_unroll=cfg["model_params"]["detach_unroll"],
    warmup_num_frames=cfg["model_params"]["warmup_num_frames"],
    discount_factor=cfg["model_params"]["discount_factor"],
)

model_path = "/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/tmpurban_driver/Urban Driver-train_flag_0signal_scene_13-il_weight_1.0-pred_weight_1.0-1/iter_0773000.pt"
model.load_state_dict(torch.load(model_path))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=model.to(device)
model = model.eval()
torch.set_grad_enabled(False)

In [ ]:
#baseline urban_driver
# model_path = "/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/examples/urban_driver/MS.pt"
model_path = "/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/tmpurban_driver/Urban Driver-train_flag_0signal_scene_13-il_weight_1.0-pred_weight_1.0-1/iter_0332000.pt"
model=torch.load(model_path)
model = model.eval()
torch.set_grad_enabled(False)

In [ ]:
#baseline urban_driver_without_BPTT
model_path = "/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/examples/urban_driver/BPTT.pt"
model=torch.load(model_path)
model = model.eval()
torch.set_grad_enabled(False)

In [ ]:
#baseline open loop torch_static
weights_scaling = [1.0, 1.0, 1.0]
_num_predicted_frames = cfg["model_params"]["future_num_frames"]
_num_predicted_params = len(weights_scaling)
import torch.nn as nn

model = VectorizedModel(
            history_num_frames_ego=cfg["model_params"]["history_num_frames_ego"],
            history_num_frames_agents=cfg["model_params"]["history_num_frames_agents"],
            num_targets=_num_predicted_params * _num_predicted_frames,
            weights_scaling=weights_scaling,
            criterion=nn.L1Loss(reduction="none"),
            global_head_dropout=cfg["model_params"]["global_head_dropout"],
            disable_other_agents=cfg["model_params"]["disable_other_agents"],
            disable_map=cfg["model_params"]["disable_map"],
            disable_lane_boundaries=cfg["model_params"]["disable_lane_boundaries"],
        )

model_path = "/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/tmpopen_loop_planner/Open Loop Planner-train_flag_0signal_scene_13-il_weight_1.0-pred_weight_1.0-1/iter_1473000.pt"
model.load_state_dict(torch.load(model_path))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=model.to(device)
model = model.eval()
torch.set_grad_enabled(False)

In [ ]:
#baseline open loop
model_path = "/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/examples/urban_driver/OL.pt"
model=torch.load(model_path)
model = model.eval()
torch.set_grad_enabled(False)

In [ ]:
#baseline open loop with history
model_path = "/mnt/share_disk/user/xijinhao/l5kit-model-based-offline-rl/examples/urban_driver/OL_HS.pt"
model=torch.load(model_path)
model = model.eval()
torch.set_grad_enabled(False)

## Load the evaluation data
Differently from training and open loop evaluation, this setting is intrinsically sequential. As such, we won't be using any of PyTorch's parallelisation functionalities.

In [4]:
# ===== INIT DATASET
eval_cfg = cfg["val_data_loader"]
eval_zarr = ChunkedDataset(dm.require(eval_cfg["key"])).open()
vectorizer = build_vectorizer(cfg, dm)
eval_dataset = EgoDatasetVectorized(cfg, eval_zarr, vectorizer)
print(eval_dataset)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


## Define some simulation properties
We define here some common simulation properties such as the length of the simulation and how many scene to simulate.

**NOTE: these properties have a significant impact on the execution time. We suggest you to increase them only if your setup includes a GPU.**

In [5]:
num_scenes_to_unroll = 10
num_simulation_steps = 248

# Closed-loop simulation

We define a closed-loop simulation that drives the SDV for `num_simulation_steps` steps while using the log-replayed agents.

Then, we unroll the selected scenes.
The simulation output contains all the information related to the scene, including the annotated and simulated positions, states, and trajectories of the SDV and the agents.  
If you want to know more about what the simulation output contains, please refer to the source code of the class `SimulationOutput`.

In [6]:
# ==== DEFINE CLOSED-LOOP SIMULATION
sim_cfg = SimulationConfig(use_ego_gt=False, use_agents_gt=True, disable_new_agents=True,
                           distance_th_far=500, distance_th_close=50, num_simulation_steps=num_simulation_steps,
                           start_frame_index=0, show_info=True)

sim_loop = ClosedLoopSimulator(sim_cfg, eval_dataset, device, model_ego=model, model_agents=None)

In [7]:
# ==== UNROLL
# scenes_to_unroll = list(range(0, len(eval_zarr.scenes), len(eval_zarr.scenes)//num_scenes_to_unroll))
scenes_to_unroll=list(range(15))
sim_outs = sim_loop.unroll([13])

/mnt/share_disk/user/xijinhao/l5kit/l5kit/l5kit/data/zarr_dataset.py:213: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  dataset = ChunkedDataset("")
/mnt/share_disk/user/xijinhao/l5kit/l5kit/l5kit/simulation/utils.py:107: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  new_dataset = ChunkedDataset("")


  0%|          | 0/248 [00:00<?, ?it/s]

TypeError: 'axis' is an invalid keyword argument for sum()

# Closed-loop metrics

**Note: for a detailed explanation of CLE metrics, please refer again to our [planning notebook](../planning/closed_loop_test.ipynb)**

In [ ]:
metrics = [DisplacementErrorL2Metric(),
           DistanceToRefTrajectoryMetric(),
           CollisionFrontMetric(),
           CollisionRearMetric(),
           CollisionSideMetric()]

validators = [RangeValidator("displacement_error_l2", DisplacementErrorL2Metric, max_value=30),
              RangeValidator("distance_ref_trajectory", DistanceToRefTrajectoryMetric, max_value=4),
              RangeValidator("collision_front", CollisionFrontMetric, max_value=0),
              RangeValidator("collision_rear", CollisionRearMetric, max_value=0),
              RangeValidator("collision_side", CollisionSideMetric, max_value=0)]

intervention_validators = ["displacement_error_l2",
                           "distance_ref_trajectory",
                           "collision_front",
                           "collision_rear",
                           "collision_side"]

cle_evaluator = ClosedLoopEvaluator(EvaluationPlan(metrics=metrics,
                                                   validators=validators,
                                                   composite_metrics=[],
                                                   intervention_validators=intervention_validators))

# Quantitative evaluation

We can now compute the metric evaluation, collect the results and aggregate them.

In [ ]:
cle_evaluator.evaluate(sim_outs)
validation_results = cle_evaluator.validation_results()
agg = ValidationCountingAggregator().aggregate(validation_results)
cle_evaluator.reset()

## Reporting errors from the closed-loop

We can now report the metrics and plot them.

In [ ]:
fields = ["metric", "value"]
table = PrettyTable(field_names=fields)

values = []
names = []

for metric_name in agg:
    table.add_row([metric_name, agg[metric_name].item()])
    values.append(agg[metric_name].item())
    names.append(metric_name)

print(table)

plt.bar(np.arange(len(names)), values)
plt.xticks(np.arange(len(names)), names, rotation=60, ha='right')
plt.show()

# Qualitative evaluation

## Visualise the closed-loop

We can visualise the scenes we have obtained previously. 

**The policy is now in full control of the SDV as this moves through the annotated scene.**

In [ ]:
output_notebook()
mapAPI = MapAPI.from_cfg(dm, cfg)
for sim_out in sim_outs: # for each scene
    vis_in = simulation_out_to_visualizer_scene(sim_out, mapAPI)
    show(visualize(sim_out.scene_id, vis_in))